# Agentic Reflection Pattern 



In [1]:
from dotenv import load_dotenv
from IPython.display import display, Markdown, HTML
from pydantic import BaseModel, Field
from typing  import Literal
from agents import (

    Agent,
    Runner,
    RunConfig,
    trace,
    ItemHelpers,
    TResponseInputItem
)
load_dotenv()

from helpers.trace_util import get_trace_url
from agents.extensions.visualization import draw_graph


In [2]:
from helpers.model_client import (
    get_openai_client,
    get_github_model_provider
)

model_name = "gpt-4o-mini"

GITHUB_MODEL_PROVIDER = get_github_model_provider(
    client = get_openai_client(),
    model = model_name
)


# Defining the agents

Here we define two agents:
- Genrator Agent
- Reviewer  or evaluator

In [3]:
class GeneratedContent(BaseModel):
    content: str = Field(description="The generated content for the presentation outline.")
    commentarytoevaluator: str = Field(description="Commentary to the evaluator about the generated content.")

generator_agent = Agent(
    name="generator_agent",
    model=model_name,
    instructions=(
        "You are a presentation outline generator agent. "
        "You will be provided with a prompt with presentation topic and audience. "
        "Your task is to generate a detailed outline for the presentation. "
        "If the prompt is not clear, ask clarifying questions to get more information. "
        "Always handoff to a single agent at a time."
    ),
    output_type=GeneratedContent
)

In [22]:
class EvaluationFeedback(BaseModel):
    feedback: str = Field(
        description="Feedback on the generated outline, including strengths and weaknesses."
    )
    score: Literal["pass", "needs_improvement", "fail"] = Field(
        description="Evaluation score for the generated outline."
        "pass: The outline meets the requirements and is well-structured. "
        "needs_improvement: The outline is acceptable but requires some adjustments. "
        "fail: The outline does not meet the requirements or is poorly structured."
    )

evaluation_agent = Agent(
    name="evaluation_agent",
    model=model_name,
    instructions=(
        "You are an evaluation agent. "
        "You will be provided with a presentation outline. "
        "Your task is to evaluate the outline based on the prompt and provide feedback "
        "and a score indicating whether the outline meets the requirements."
        "The feedback should include strengths and weaknesses of the outline, "
        "If it's not good enough, you provide feedback on what needs to be improved."
        "You dont seek perfection, you seek a good enough outline that demostrates"
        " thoughtful effort and ofcourse improvement. "
        "Never give it a pass on the first try."
        "also dont be too harsh, pass after a few tries if there is improvement"

    ),
    output_type=EvaluationFeedback,
)

# Running the Reflection Workflow 

1. using a deterministic flow managed in code 
2. using a stochastic flow managed by the LLM: let the LLM decide the next step




## Deterministic flow managed in code

Since we are managing the flow in code, we can use the `max_turns` parameter to limit the number of turns. 

This is useful for debugging and testing purposes. 

```python
inspect the use of max turns


In [9]:
msg = input("Enter the prompt for the presentation outline and audience: ") or (
            "Generate a presentation outline for a technical audience on"
             "the topic of AI in healthcare.")

input_items: list[TResponseInputItem] = [{"content": msg, "role": "user"}]

latest_output : str | None = None
latest_feedback : EvaluationFeedback | None = None

i = 0
turn = 0
max_turns = 3

with trace(workflow_name="reflection_workflow", group_id="mdsi") as tr:
    print(f"Tracing url: {get_trace_url(tr)}")

    while True:

        generated_outline = await Runner.run(
            generator_agent,
            input_items,
            run_config=RunConfig(
                model=model_name,
                model_provider=GITHUB_MODEL_PROVIDER,
            )
        )

        # append the generation output to the message history
        input_items.append(generated_outline.to_input_list()[-1])


        generated_result : GeneratedContent = generated_outline.final_output
        latest_output = generated_result.content
        print(f"\n++ Commentary: {generated_result.commentarytoevaluator}")

        # invoke the evaluation agent with the generated outline
        evaluation_result = await Runner.run(
            evaluation_agent,
            input_items,
            run_config=RunConfig(
                model=model_name,
                model_provider=GITHUB_MODEL_PROVIDER,
            ),
            max_turns=3,
        )

        # increment the turns count for the evaluation cycle
        turn += 1

        # inspect the evaluation agent feedback
        latest_feedback: EvaluationFeedback = evaluation_result.final_output
        print(f">> Evaluation Feedback (turn: {turn}): ", latest_feedback.feedback)
        print(">> Evaluation Score: ", latest_feedback.score)

        # determin if to break the loop
        if latest_feedback.score == "pass":
            print(":: Outline is acceptable. Ending the loop.")
            break

        if i >= max_turns:
            print("!! Maximum iterations reached. Ending the loop.")
            break
        i += 1

        print(">< Outline didn't pass. Re-run with feedback.")
        input_items.append(
            {
                "role": "user",
                "content": f"Feedback: {latest_feedback.feedback}"
            }
        )





Tracing url: https://platform.openai.com/traces/trace?trace_id=trace_6a251628e50b4999b2bc9d9fdd813e9b

++ Commentary: The outline presents a comprehensive overview of the topic of AI in healthcare tailored for a technical audience. It includes detailed sections on current applications, enabling technologies, case studies, and addresses the challenges faced in the field. The outline ends with future trends and aims to engage the audience in a Q&A session, promoting further discussion. The references section is also included to guide the audience towards further learning, which is crucial for a technical presentation.
>> Evaluation Feedback (turn: 1):  ### Feedback
- **Strengths:**  
   - The outline is well-structured and comprehensive, covering key areas of AI in healthcare such as current applications, technologies, case studies, challenges, and future trends.  
   - The inclusion of case studies adds practical examples that enhance understanding.  
   - The outline caters to a techni

In [10]:
display(Markdown(latest_output))

### Enhanced Presentation Outline: AI in Healthcare

#### I. Introduction  
   A. Definition of Artificial Intelligence (AI)  
   B. Significance of AI in Healthcare  
      1. Impact on Global Healthcare Challenges: e.g., Rising Costs and Accessibility  
      2. Highlight a statistic: Improvement in diagnostic accuracy or costs reduction due to AI 
   C. Audience Considerations  
      1. Expected level of prior knowledge: Familiarity with AI concepts, basic data science, and healthcare industry terminology  
      2. Importance of understanding the applications and limitations of AI technology  

#### II. Current Applications of AI in Healthcare  
   A. Diagnostic Tools  
      1. Advanced Imaging Analysis (e.g., Radiology, Pathology)  
         - Brief on AI algorithms applied  
      2. Predictive Analytics for Early Disease Detection  
   B. Personalized Medicine  
      1. Genomic Data Analysis for Tailored Treatments  
      2. AI's Role in Predicting Drug Responses  
   C. Virtual Health Assistants  
      1. Patient Interaction Tools (e.g., Chatbots)  
      2. AI Applications in Appointment and Medication Reminders  
   D. Drug Discovery and Development  
      1. How AI Identifies Drug Candidates  
      2. Enhancing Clinical Trial Efficiency Through AI  

#### III. Key Technologies Powering AI in Healthcare  
   A. Overview of Machine Learning Algorithms  
      1. Differences Between Supervised and Unsupervised Learning  
      2. Specific Use Cases in Healthcare  
   B. Natural Language Processing (NLP)  
      1. Applications in Clinical Documentation and Note Analysis  
      2. Example of Structuring Data from Electronic Health Records  
   C. Deep Learning Techniques  
      1. CNNs in Imaging Analysis  
      2. RNNs for Monitoring Patient Vital Signs Over Time  

#### IV. Practical Case Studies  
   A. Success Stories in AI Implementation  
      1. Case Study: AI Enhancements in Imaging Diagnostics (e.g., lung cancer detection)  
      2. Case Study: AI in Oncology and Personalized Treatment Plans  
   B. Applications in Remote Patient Monitoring  
      1. Data Analysis from Wearable Device Usage  
      2. Observations From the COVID-19 Pandemic and Remote Care  

#### V. Challenges and Ethical Considerations  
   A. Data Privacy and Security Challenges  
   B. Addressing Algorithmic Bias  
      1. Real-life Examples of Bias in Health Predictions  
   C. Ethical and Regulatory Standards  
      1. Need for Guidelines: Transparency and Accountability  
      2. Case References of Ethical Dilemmas Encountered with AI Adoption  
   D. System Integration Issues  
      1. Challenges in Merging AI with Existing Healthcare Infrastructure  

#### VI. Future Trends in AI for Healthcare  
   A. Innovations in Predictive and Preventive Healthcare  
   B. The Role of Personalization in Future Treatments  
      1. Emergence of Tailored Health Plans  
   C. Growing Interoperability Among Diverse Healthcare AI Systems  
   D. Continuing Ethical Discourse and Governance Development  

#### VII. Conclusion  
   A. Recap of Key Insights and Takeaways  
   B. Vision for AI's Future in Transforming Healthcare Delivery  
   C. Call to Action: Collaboration Towards Ethical AI Development and Implementation  

#### VIII. Q&A Session  
   A. Engage the Audience with Open Questions  
   B. Discussions on Experiences and Aspirations for AI in Healthcare  

#### IX. References  
   A. Cite Recent Research Studies and Articles for Further Reading  
   B. Additional Resources for Continued Learning Available  
   C. Acknowledge Contributions from the Field  

### Additional Notes  
- Plan to engage with the audience by referencing current news or advancements related to healthcare AI.  
- Use visuals (graphs, images) sparingly but effectively to illustrate key points without overcrowding slides.  
- Remain adaptable during the presentation to address audience interest areas as they arise.

## Stochastic flow managed by the LLM

In this case we define an orchestrator agent, with tools to generate and evaluate the text.

The tools are the same agents that were used before in the deterministic flow. Except that in this case we handover the control to the LLM.



### Defining the orchestrator agent 


key components of the orchestrator agent:
- tools: the tools are the same agents that were used before in the deterministic flow. Except that in this case we handover the control to the LLM.
- llm: the LLM is the same as before, but we can use a model  if we want to. Use a powerful model for the orchestrator agent, since it will be the one that decides the next step.
- instructions: your system prompt for the orchestator, this is the most important part of the agent. The instructions should be clear and concise, and should include the following:
  - the goal of the agent
  - the tools available to the agent
  - any other relevant information

In [23]:
reflection_orchestrator_agent = Agent(
    name="reflection_orchestrator_agent",
    model=model_name,
    instructions=(
        "You are a reflection orchestrator agent. "
        "You will be provided with a prompt with presentation topic and audience. "
        "Leverage the specialised skills in the available tools to complete this task"
    ),
    tools=[
        generator_agent.as_tool(
            tool_name="outline_generator",
            tool_description="Generates a detailed outline for the presentation.",
            custom_output_extractor=None

        ),
        evaluation_agent.as_tool(
            tool_name="outline_evaluator",
            tool_description="Evaluates the generated outline and provides feedback.",
            custom_output_extractor=None
        )
    ],
    tool_use_behavior="run_llm_again",

)

try:
    draw_graph(reflection_orchestrator_agent, filename="viz/10_reflection_as_tools.gv")
except Exception as e:
    print(f"An error occurred: {e}")

An error occurred: failed to execute WindowsPath('dot'), make sure the Graphviz executables are on your systems' PATH


### Adding hooks 

Run hooks allow you to execute code at different stages of the agent or run lifecycle.

In [24]:
from agents import RunContextWrapper, RunHooks, MaxTurnsExceeded

class ReflectionContext:
    turns: int = 0
    max_turns: int = 3

class ReflectionHooks(RunHooks):
    def __init__(self, temp_outside: int):
        super().__init__()
        self.temp_outside = temp_outside # can be used as a constant as it doesnot really maintain state

    def on_agent_end(self, context, agent, output):
        print(f"agent name : {agent.name}")
        print(f"state pass from outside : {self.temp_outside}")

        reflectionContext : ReflectionContext = context.context
        reflectionContext.turns += 1 # this is how you would manage context between turns
        print(f"context turns : {reflectionContext.turns}")

        if reflectionContext.turns >= reflectionContext.max_turns:
            print("!! Maximum iterations reached. Ending the loop.")
            raise MaxTurnsExceeded(f"This reflection agent has reached its turns limit of {reflectionContext.max_turns}")

        return super().on_agent_end(context, agent, output)


### Run the Reflection Planner 

In [25]:

with trace(workflow_name="reflection_as_tools_workflow", group_id="mdsi") as tr:
    print(f"Tracing url: {get_trace_url(tr)}")

    msg = input ("Enter your message: ") or "How to prepare a presentation outline? for post grad students"

    input_items: list[TResponseInputItem] = [{"content": msg, "role": "user"}]

    reflection_result = None
    reflection_context = ReflectionContext() # default max truns == 3
    some_app_state_var = 25 # to demostator passing state of app to agents in workflow
    while True:
        try:
            reflection_result = await Runner.run(
                reflection_orchestrator_agent,
                input_items,
                run_config=RunConfig(
                    model=model_name,
                    model_provider=GITHUB_MODEL_PROVIDER,

                ),
                context=reflection_context,  ## <-- I am managing the number of reflection turns in context (short term memory), increments everytime an agent ends
                hooks=ReflectionHooks(temp_outside=some_app_state_var), ## <-  pass some value from own mnaged state in the app
                max_turns=10 # <-- Runner uses the max_turns to abort, this one includes any tool invocaitions
            )

            # change app state =
            some_app_state_var = some_app_state_var * 1.02
        except MaxTurnsExceeded as e:
            print(f"!! Maximum iterations reached. Ending the loop. {e}")
            break
        finally:
            print("finally block executed")
            print(reflection_result)

    print(reflection_result)

Tracing url: https://platform.openai.com/traces/trace?trace_id=trace_c350955369974a25ba96d6431545c3ab
agent name : reflection_orchestrator_agent
state pass from outside : 25
context turns : 1
finally block executed
RunResult:
- Last agent: Agent(name="reflection_orchestrator_agent", ...)
- Final output (str):
    Here is a detailed outline on how to prepare a presentation, tailored for post-graduate students:
    
    ### Presentation Outline Preparation
    
    #### 1. Define the Purpose of the Presentation
       - **Identify the main goal:** What do you want to achieve?
         - Inform?
         - Persuade?
         - Teach?
       - **Understand your audience:** Who are they and what do they expect?
    
    #### 2. Research Your Topic
       - **Gather relevant information:**
         - Books
         - Articles
         - Online resources
       - **Identify key points:** What are the most important aspects of your topic?
    
    #### 3. Create a Thesis Statement
       - **S